# Lab 1: LoRA & QLoRA - Fine-Tuning a Llama-2 Model
---
## Notebook 2: The Training Process

**Goal:** This is the core of the lab. You will load a quantized base model and a dataset, configure LoRA, and then launch the training job using the `transformers` Trainer.

**You will learn to:**
-   Use `BitsAndBytesConfig` to load a model in 4-bit precision (QLoRA).
-   Load a dataset and prepare it for training.
-   Deeply understand the parameters of `peft.LoraConfig` to control how LoRA is applied.
-   Use `peft.get_peft_model` to create a trainable PEFT model.
-   Configure `transformers.TrainingArguments` for the fine-tuning task.
-   Use the `transformers.Trainer` to run the fine-tuning process.


### Step 1: Load Model and Tokenizer

First, we need to load our base model. We'll use `meta-llama/Llama-2-7b-chat-hf`. To make this large model fit into memory on a single consumer GPU, we'll use **4-bit quantization** via the `bitsandbytes` library.

#### Key Hugging Face Components:

-   `torch`: We set the default tensor type to `bfloat16`, a datatype optimized for deep learning that offers a good balance between range and precision.
-   `transformers.BitsAndBytesConfig`: This is the configuration class that enables quantization.
    -   `load_in_4bit=True`: Activates 4-bit loading.
    -   `bnb_4bit_quant_type="nf4"`: Specifies the "NormalFloat4" (nf4) quantization type, which is particularly effective for models with normally distributed weights, like LLMs.
    -   `bnb_4bit_compute_dtype=torch.bfloat16`: While the model weights are stored in 4-bit, computations (like matrix multiplications during the forward and backward passes) are performed in 16-bit `bfloat16` for accuracy and stability. This is the essence of **QLoRA**.
-   `transformers.AutoModelForCausalLM`: A generic model class that automatically loads the correct architecture for causal language models (like Llama-2).
    -   `from_pretrained()`: The primary method for loading models from the Hugging Face Hub.
    -   `quantization_config`: We pass our `BitsAndBytesConfig` here to apply quantization on-the-fly during model loading.
    -   `device_map="auto"`: This tells `accelerate` to automatically figure out the best way to distribute the model across available hardware (e.g., placing all of it on the GPU if it fits).
-   `transformers.AutoTokenizer`: Loads the correct tokenizer for a given model.
    -   `tokenizer.pad_token = tokenizer.eos_token`: For autoregressive models like Llama, there's often no dedicated padding token. A common practice is to use the End-Of-Sentence (`eos`) token for padding.


In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "NousResearch/Llama-2-7b-hf"

# --- Set Default Tensor Type ---
# 使用 bfloat16 作為預設張量類型，這是一種針對深度學習優化的資料類型
# 在保持數值範圍的同時提供適當的精度，特別適合大型語言模型
torch.set_default_dtype(torch.bfloat16)

# --- Quantization Configuration ---
# This is the core of QLoRA
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# --- Load Tokenizer ---
# Note: You'll need to have logged in to your Hugging Face account
# using `huggingface-cli login` and have access to the Llama-2 model.

# --- Set Cache Directory ---
# Specify the download folder for pretrained models
# The path r'D:\Pretrained_models\modelscope' is a Windows path.
# Since you are running in a WSL environment, you need to use the Linux path format.
# Windows drives are typically mounted under /mnt/ in WSL.
# For example, D:\ would become /mnt/d/.
# Please modify the path below to match your actual directory in WSL.

cache_dir = "/mnt/d/Pretrained_models/modelscope"

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True, cache_dir=cache_dir)
tokenizer.pad_token = tokenizer.eos_token

# --- Load Model ---
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="auto",
)

print("✅ Model and Tokenizer loaded successfully!")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Model and Tokenizer loaded successfully!


### Step 2: Configure LoRA with `peft`

Now we apply the LoRA adapter to our quantized model.

#### Key Hugging Face `peft` Components:

-   `peft.LoraConfig`: This class holds all the configuration parameters for LoRA.
    -   `r`: The **rank** of the update matrices. This is the most critical LoRA hyperparameter. It determines the number of trainable parameters. A higher `r` means more expressive power but also more parameters. A common range is 8-64. We'll use 16.
    -   `lora_alpha`: The scaling factor for the LoRA updates. It's related to the learning rate. A common practice is to set `lora_alpha` to be double the `r` value.
    -   `target_modules`: This is a list of the names of the modules (layers) in the base model that we want to apply LoRA to. For Llama models, this is typically the query (`q_proj`) and value (`v_proj`) projection matrices in the self-attention layers. Applying it to more layers can improve performance but increases trainable parameters.
    -   `lora_dropout`: A standard dropout probability applied to the LoRA layers to prevent overfitting.
    -   `bias="none"`: Specifies which biases to train. `"none"` means no biases are trained, which is standard practice for LoRA.
    -   `task_type="CAUSAL_LM"`: Informs `peft` that we are performing a Causal Language Modeling task.
-   `peft.get_peft_model`: This function takes the base model and the `LoraConfig` and returns a new `PeftModel`. This new model wraps the original, freezing all its weights and injecting the trainable LoRA adapters into the specified `target_modules`.

After creating the `PeftModel`, we can use `print_trainable_parameters()` to see just how effective LoRA is at reducing the number of parameters we need to update.


In [2]:
from peft import LoraConfig, get_peft_model

# --- LoRA Configuration ---
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# --- Create PeftModel ---
# This freezes the base model and adds the LoRA adapters
peft_model = get_peft_model(model, lora_config)

# --- Print Trainable Parameters ---
# See how few parameters we'll actually be training!
peft_model.print_trainable_parameters()


trainable params: 8,388,608 || all params: 6,746,804,224 || trainable%: 0.1243


### Step 3: Load Dataset and Set Up Training

The final step is to configure and run the training process using the `transformers.Trainer`.

#### Key Hugging Face Components:

-   `datasets.load_dataset`: Loads the `mlabonne/guanaco-llama2-1k` dataset, which contains 1,000 instruction-response pairs formatted for Llama-2.
-   `transformers.TrainingArguments`: A comprehensive class to define all training hyperparameters.
    -   `output_dir`: Where to save model checkpoints and logs.
    -   `per_device_train_batch_size`: The number of samples per batch on each GPU.
    -   `gradient_accumulation_steps`: Simulates a larger batch size. The effective batch size is `batch_size * grad_accumulation_steps`. This is useful for fitting larger batches into limited GPU memory.
    -   `learning_rate`: The learning rate for the optimizer (AdamW by default).
    -   `num_train_epochs`: The total number of times to iterate over the dataset.
    -   `logging_steps`: How often to log training progress.
    -   `fp16=True`: Enables automatic mixed-precision training, which speeds up training and reduces memory usage without significant loss in precision.
-   `transformers.Trainer`: This class orchestrates the entire training loop. It handles batching, optimization, logging, and saving checkpoints.
    -   We provide it with our `peft_model`, the `training_args`, the `train_dataset`, and a `data_collator`.
    -   `data_collator`: A function that takes a list of samples from the dataset and collates them into a single batch tensor. `DataCollatorForLanguageModeling` is used for language modeling tasks.

With everything configured, calling `trainer.train()` kicks off the fine-tuning process.


In [3]:
import transformers
from datasets import load_dataset

# --- Load Dataset ---
dataset_name = "mlabonne/guanaco-llama2-1k"
# The dataset is small, so we'll use the whole thing for training.
dataset = load_dataset(dataset_name, split="train", cache_dir="/mnt/d/Pretrained_models/modelscope")

# --- Preprocessing ---
# We need to tokenize our dataset so the model can understand it.
def preprocess_function(examples):
    # The guanaco dataset format is a single 'text' field.
    # We will tokenize it and set the labels to be the same as the input_ids
    # for the causal language modeling task.
    tokenized_inputs = tokenizer(
        examples["text"], 
        truncation=True, 
        padding="max_length", 
        max_length=512 # You can adjust this based on your VRAM and typical sequence length
    )
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"]
    return tokenized_inputs

print("Preprocessing dataset...")
# We use remove_columns to get rid of the original 'text' column, which the Trainer doesn't need.
tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["text"])
print("✅ Dataset preprocessed.")


# --- Training Arguments ---
training_args = transformers.TrainingArguments(
    output_dir="./lora-llama2-7b-guanaco",
    per_device_train_batch_size=4, # Reduced from 16 to 4 to prevent potential OOM errors
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=1,
    logging_steps=10,
    fp16=True,  # Use mixed precision
    save_total_limit=2,
    overwrite_output_dir=True,
)

# --- Create Trainer ---
# We pass the tokenized_dataset to the trainer.
trainer = transformers.Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

# A known issue with gradient checkpointing and PEFT requires this
peft_model.config.use_cache = False

# --- Start Training ---
print("🚀 Starting training...")
trainer.train()
print("✅ Training complete!")


Preprocessing dataset...
✅ Dataset preprocessed.
[2025-08-22 00:50:29,228] [INFO] [real_accelerator.py:260:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


[2025-08-22 00:50:31,127] [INFO] [logging.py:107:log_dist] [Rank -1] [TorchCheckpointEngine] Initialized with serialization = False
🚀 Starting training...


Step,Training Loss
10,1.431200
20,1.329600
30,1.311800
40,1.214000
50,1.229400
60,1.186600


✅ Training complete!
